### Configure EIGRP From a File.

Another powerful ability in Netmiko is the ability to deploy configuration using a text file.

This could be a straight text file that you could create but what is the fun in that?

[Jinja](https://jinja.palletsprojects.com/en/2.11.x/) is a powerful templating language that is widely used.

In this lab we are going to utilize jinja to create a configuration file, and push that file using Netmiko.

In order to utilize the templating function of jinja we need to import it utilizing ```from jinja2 import Template```


In [1]:
import yaml
from jinja2 import Template
from getpass import getpass
from netmiko import Netmiko
from pprint import pprint

In [2]:
def read_inventory(inventory_file):
    with open(inventory_file, 'r') as f:
        inventory = yaml.safe_load(f)
    return inventory

### Creating Config Snippet

This function is going to take our jinja template and utilize the information read in from our inventory file to render a string.

The jinja template:

```jinja2
router eigrp {{ eigrp.as }}
{% for eigrp_network in eigrp.advertised_networks -%}
network {{ eigrp_network.network }} {{ eigrp_network.prefix }}
{% endfor -%}
{% if router_id is defined -%}
router-id {{ router_id }}
{% endif -%}

```

Example data from a router:
```python
{'host': '10.246.32.203', 'port': 1222, 'device_type': 'cisco_ios', 'eigrp': {'as': 1, 'advertised_networks': [{'network': '1.1.1.1', 'prefix': '0.0.0.0'}, {'network': '192.168.100.0', 'prefix': '0.0.0.255'}]}, 'interfaces': [{'name': 'loopback1', 'address': '1.1.1.1', 'netmask': '255.255.255.255', 'enabled': True}]}
```

Jinja will go through the template and match up dictionary and populate the necessary fields.

For example:
```jinja2
router eigrp {{ eigrp.as }}
```
Will grab the ```eigrp['as]``` and this will give us ```router eigrp 1```.

Jinja can also perform loops with the ```{% for _ in __ %} {% endfor %}``` logic. 

This will read through our advertised_networks list of dictionaries and populate the correct information:
```
network 1.1.1.1 0.0.0.0
network 192.168.100.0 0.0.0.255
```

Jinja also allows for checking if a variable exists by the ```{% if __ %} {% endif %}``` along with more complex if, else statements. In our above sample data we do not have a router_id defined in our inventory. However instead of erroring out, Jinja skips it and proceeds.

In [3]:
def generate_config_snippet(jinja_template, value):
    with open(jinja_template) as f:
        template = Template(f.read())
        config_snippet = template.render(value)
    return config_snippet

### Write String to File

With our config_snippet string generated from Jinja we can easily write this to a specific output file. 

In [4]:
def write_config_snippet(config_snippet, output_file):
    with open(output_file, 'w') as f:
              f.write(config_snippet)

In [5]:
if __name__ == '__main__':
    username=input('Enter your username: ')
    password=getpass('Enter your password: ')

Enter your username: admin
Enter your password: ········


In [6]:
    inventory_file = 'inventory/routers.yml'
    inventory = read_inventory(inventory_file)

### Push configuration to devices

Using our inventory file we can easily use Netmiko's send_config_from_file method to push our jinja generated configuration to our devices.

In [7]:
    jinja_template = 'templates/eigrp.j2'
    for key, value in inventory.items():
        output_file = f"snippets/{key}_snippet.cfg"
        config_snippet = generate_config_snippet(jinja_template, value)
        write_config_snippet(config_snippet, output_file)
        with Netmiko(host=value['host'], 
                     port=value['port'],
                     username=username, 
                     password=password, 
                     device_type=value['device_type']) as session:
            print(session.find_prompt())
            output = session.send_config_from_file(output_file)
            print(output)

CSR1#
config term
Enter configuration commands, one per line.  End with CNTL/Z.
CSR1(config)#router eigrp 1
CSR1(config-router)#network 1.1.1.1 0.0.0.0
CSR1(config-router)#network 192.168.100.0 0.0.0.255
CSR1(config-router)#end
CSR1#
CSR3#
config term
Enter configuration commands, one per line.  End with CNTL/Z.
CSR3(config)#router eigrp 1
CSR3(config-router)#network 3.3.3.3 0.0.0.0
CSR3(config-router)#network 192.168.100.0 0.0.0.255
CSR3(config-router)#end
CSR3#
